# 언어 모형

## 학습 목표

- 언어 모형
- n-gram
- Byte Pair Encoding

## 언어 모형

- language models
- 한 자연어 문장의 확률
- 예:
  - A) 나는 밥을 먹었다
  - B) 나는 밥을 마셨다
- A)가 B)보다 확률이 높음

## 언어 모형의 중요성

- 문장 생성에 사용 (나는 밥을 ~)
  - 기계 번역
  - 오타 교정
- 사전 학습(pretraining)
  - 문장의 다음 단어를 예측하도록 학습
  - 이후 본 과제에 학습

## 조건부 확률

- P(B|A): A가 주어졌을 때 B의 확률
- P(먹었다|밥): '밥'이 주어졌을 때 '먹었다'의 확률
- P(마셨다|밥): '밥'이 주어졌을 때 '마셨다'의 확률

## 연쇄 규칙 (1)

$$
P(A, B) = P(A) \times P(B|A)
$$

## 연쇄 규칙 (2)

$$
\begin{align}
P(A, B, C) &= P(A, B) \times P(C|A, B) \\
           &= P(A) \times P(B|A) \times P(C|A, B)
\end{align}
$$

## 독립

- A와 B가 독립이면 $P(A, B) = P(A) \times P(B)$
- 연쇄규칙에 따라 $P(A, B) = P(A)P(B|A)$이므로
- A와 B가 독립이면 $P(B|A) = P(B)$도 성립

## n-gram

- 가장 단순한 형태의 언어 모형
- 문장을 n개의 토큰 조합으로 이해
  - 토큰(token): 분석의 단위. 글자 또는 단어 등

## 유니그램
  - uni = 1
  - 문장의 모든 토큰이 독립
  - P(오늘 밥 먹었다) = P(오늘) \* P(밥) \* P(먹었다) 
  - 토큰의 통계 = 문장의 통계

## 실습 데이터

데이터 다운로드

In [1]:
import requests

In [2]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip'
res = requests.get(url)
with open('sentiment.zip', 'wb') as f:
    f.write(res.content)

## 압축 열기

In [3]:
from zipfile import ZipFile

In [4]:
z = ZipFile('sentiment.zip')

In [5]:
for file in z.filelist:
    print(file.filename)

sentiment labelled sentences/
sentiment labelled sentences/.DS_Store
__MACOSX/
__MACOSX/sentiment labelled sentences/
__MACOSX/sentiment labelled sentences/._.DS_Store
sentiment labelled sentences/amazon_cells_labelled.txt
sentiment labelled sentences/imdb_labelled.txt
__MACOSX/sentiment labelled sentences/._imdb_labelled.txt
sentiment labelled sentences/readme.txt
__MACOSX/sentiment labelled sentences/._readme.txt
sentiment labelled sentences/yelp_labelled.txt
__MACOSX/._sentiment labelled sentences


## 영화평 불러오기

In [6]:
import pandas as pd

In [7]:
f = z.open('sentiment labelled sentences/imdb_labelled.txt')
movie = pd.read_csv(f, sep='\t', header=None)

## 영화평 보기

In [8]:
movie.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


## 컬럼명 수정

In [9]:
movie.columns = ['review', 'sentiment']

In [10]:
movie.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


## 단어 문서 행렬

- 단어 문서 행렬(Term Document Matrix)
- 문서별로 각 단어가 사용된 횟수를 정리한 표
- 행(문서), 열(단어)

## CountVectorizer

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
cv = CountVectorizer()

In [13]:
tdm = cv.fit_transform(movie['review'])

In [14]:
tdm.shape

(748, 3047)

## 단어 추출

In [15]:
words = cv.get_feature_names()

In [16]:
words[400:410]

['cartoon',
 'cartoons',
 'case',
 'cases',
 'cast',
 'casted',
 'casting',
 'cat',
 'catchy',
 'caught']

## 단어 빈도 합계

In [18]:
tdm.sum()

13731

In [17]:
# 열별로 단어들의 합계를 더함. 
counts = tdm.sum(axis=0)

In [19]:
counts[:, 400:410]

matrix([[ 3,  2,  2,  1, 18,  1,  6,  1,  1,  1]], dtype=int64)

## 단어 사용 빈도를 표로 정리

In [20]:
wc = pd.DataFrame({'단어': words, '빈도': counts.flat})

C:\Users\inyoung\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\dtypes\dtypes.py:271: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


In [21]:
wc.sort_values('빈도').tail(10)

,단어,빈도
1748,movie,182
2917,was,186
1358,in,203
2694,to,253
2658,this,292
1428,it,325
1423,is,340
1837,of,377
125,and,434
2638,the,849


## 단어 사용 빈도 저장

In [23]:
wc.to_csv('단어빈도.csv')

## 바이그램
- bi = 2
- P(오늘 밥 먹었다) = P(오늘) \* P(밥|오늘) \* P(먹었다|밥) 

## ngram_range

- ngram의 범위를 지정 (최소, 최대)
- `(2, 2)`로 지정하면 bigram
- `(1, 2)`로 지정하면 unigram + bigram

In [30]:
bicv = CountVectorizer(ngram_range=(2,2))

In [31]:
bidm = bicv.fit_transform(movie['review'])

In [32]:
bidm.shape

(748, 9732)

## 표로 정리

In [33]:
bigrams = bicv.get_feature_names()

In [34]:
bicounts = bidm.sum(axis=0)

In [35]:
bc = pd.DataFrame({'바이그램': bigrams, '빈도': bicounts.flat})

C:\Users\inyoung\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\dtypes\dtypes.py:271: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


## 자주 나오는 바이그램

In [36]:
bc.sort_values('빈도').tail(10)

,바이그램,빈도
7904,the movie,39
5883,one of,40
662,and the,41
4389,it was,42
7814,the film,44
8277,this is,47
8265,this film,51
3941,in the,64
8287,this movie,71
5752,of the,116


## n-gram의 문제

- 실제 문장에서 단어들은 독립이 아님
- n이 작으면 부정확
- n이 크면 조합이 많아짐

## Byte Pair Encoding

- 원래는 데이터 압축 방법
- 최근 텍스트 분석에서도 활용
- 정해진 횟수만큼 다음을 반복
  1. 토큰들의 바이그램을 만든다
  2. 가장 빈도가 높은 바이그램을 하나의 토큰으로 취급한다